In [1]:
from langchain_openai import OpenAI, ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import YoutubeLoader
import streamlit as st
import os

In [3]:

OPENAI_API_KEY = st.secrets["YOUTUBE_OPENAI_API_KEY"]
youtube_url = "https://www.youtube.com/watch?v=P1iCMyuwOfg&ab_channel=ChefReactions"
youtube_url_2 = "https://www.youtube.com/watch?v=Gm2-Kc-A7Kc&t=2s&ab_channel=ColoradoChannel"
openai_embed_model = "text-embedding-ada-002"
openai_model = "gpt-3.5-turbo-16k"
llm = ChatOpenAI(api_key=OPENAI_API_KEY, model=openai_model, temperature=0.1)

In [4]:
def process_video(url):
    loader = YoutubeLoader.from_youtube_url(url)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=200)
    chunks = text_splitter.split_documents(documents)
    embedding = OpenAIEmbeddings(api_key=OPENAI_API_KEY, model = "text-embedding-ada-002")
    vector_store = FAISS.from_documents(chunks, embedding)
    return vector_store

In [5]:
vector_store = process_video(youtube_url_2)

In [6]:
query1 = 'give me a bulleted list of the talking points'
query2 = 'what is the most important point of the video?'
query3 = 'please list the bills that were voted on. Give the bill number, the bill name, and the main talking points about each'

In [7]:
def process_question(question):
    retriever = vector_store.as_retriever()
    qa = RetrievalQA.from_chain_type(llm = llm, chain_type='stuff', retriever=retriever)
    results = qa.invoke(question)
    return results


In [9]:
response = process_question(query3)

In [10]:
print(response)

{'query': 'please list the bills that were voted on. Give the bill number, the bill name, and the main talking points about each', 'result': 'Here are the bills that were voted on:\n\n1. House Bill 1368 - This bill concerns requirements for property and casualty insurers offering insurance policies in the state to provide certain documents to their insurance. It was adopted without much discussion.\n\n2. House Bill 1381 - This bill concerns the continuation of the Division of Financial Services in the Department of Regulatory Agencies. It updates fees paid to public trustees to cover the cost of certain tasks related to foreclosures. It was adopted without much discussion.\n\n3. House Bill 1438 - This bill concerns the implementation of certain affordable prescription drug programs. It ensures that Coloradans who need insulin and epipens can access them at an affordable price. It was adopted without much discussion.\n\n4. House Bill 1443 - This bill concerns fees to which public truste